# Setup

In [ ]:
import gymnasium as gym
env = gym.make('MountainCar-v0')
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from matplotlib import pyplot as plt

In [ ]:
from dqn import QNetwork, DQNAgent, ReplayBuffer

In [ ]:
# hyperparameters
learning_rate = 1e-3
n_episodes = 3_000
start_epsilon = 0.9
final_epsilon = 0.05
epsilon_decay = 0.99
# reduce the exploration over time
batch_size = 64
discount_factor = 0.99
replay_size = 10_000
logging_interval = 50
hidden_size=64
dropout_rate=0.1
weight_decay=1e-4
target_network = False
target_network_update = int(1e4)
alpha = 0
np.random.seed(42)
auxiliary = True
intermediate_reward = 1
final_reward = 100
amsgrad = False

In [ ]:
agent = DQNAgent(
    learning_rate=learning_rate,
    state_size=2,
    action_size=3,
    discount_factor=discount_factor,
    final_epsilon=final_epsilon,
    hidden_size=hidden_size,
    epsilon_decay=epsilon_decay,
    initial_epsilon=start_epsilon,
    replay_size=replay_size,
    dropout_rate=dropout_rate,
    target_network=target_network,
    weight_decay=weight_decay,
    target_network_update=target_network_update,
    alpha=alpha,
    amsgrad=amsgrad
)

In [ ]:
# run = wandb.init(project='ANN', config={"learning_rate": learning_rate, "n_episodes": n_episodes, "start_epsilon": start_epsilon, "final_epsilon": final_epsilon, "epsilon_decay": epsilon_decay, "batch_size": batch_size, "discount_factor": discount_factor, "replay_size": replay_size, "hidden_size": hidden_size, "dropout_rate": dropout_rate, "weight_decay":weight_decay, "target_network":target_network, "alpha":alpha,"target_network_update":target_network_update, "auxiliary":auxiliary, "final_reward":final_reward, "intermediate_reward":intermediate_reward, "amsgrad":amsgrad}, name='DQN')


In [ ]:
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)
with tqdm(total=n_episodes, desc=f"Episode 0/{n_episodes}") as pbar:
    losses = []
    rewards = []
    target_count = 0
    finished = []
    episode_steps = []
    for episode in tqdm(range(n_episodes)):
        obs, info = env.reset()
        done = False
        # play one episode
        t = 0
        episode_reward = 0
        episode_loss = 0
        
        while not done:
            action = agent.get_action(obs, env=env)
            next_obs, reward, terminated, truncated, info = env.step(action)

            # update if the environment is done and the current obs
            done = terminated or truncated
            if terminated:
                next_obs = (None, None)
                reward+=final_reward
            reward+=intermediate_reward*(1.8-(0.6-next_obs[0]))/1.8

            loss, target_count = agent.update(obs, action, reward, next_obs, batch_size=batch_size, target_count=target_count)
            if loss is not None:
                episode_reward += reward
                episode_loss+=loss
            obs = next_obs
            t+=1
        finished.append(terminated)
        episode_steps.append(t)
        rewards.append(episode_reward)
        losses.append(episode_loss)
        agent.decay_epsilon()
        pbar.set_description(f"Episode {episode + 1}/{n_episodes}")
        pbar.set_postfix(train_loss=loss, epsilon=agent.epsilon, target_count=target_count)
        pbar.update(1)
        pbar.refresh() 
        if episode % logging_interval == 0 and episode>100:
            # wandb.log({"train_loss": np.mean(losses), "epsilon": agent.epsilon, "episode_steps": np.mean(episode_steps), "finished": np.sum(finished), "mean_reward": np.mean(rewards)})
            losses = []
            rewards = []
            finished = []
            episode_steps = []


In [ ]:
wandb.finish()